# Merge probes for CTP-10 Aire

There are 3 different sources:
1. Aire TSSs
2. Aire intronic RNA
3. selected promoter enhancer pairs (3, 10kb sequential)

In [5]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3
from ImageAnalysis3 import get_img_info, visual_tools, corrections#, library_tools

#from ImageAnalysis3.library_tools import LibraryDesigner as ld
#from ImageAnalysis3.library_tools import LibraryTools

%matplotlib notebook
print(os.getpid())

# biopython for SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# blast
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML 
#import ImageAnalysis3.library_tools.quality_check as qc

31000


In [6]:
# Library directories
pool_folder = r'\\smirnov\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain'
sub_fds = [os.path.join(pool_folder, _fd) for _fd in os.listdir(pool_folder) if os.path.isdir(os.path.join(pool_folder, _fd))]


In [7]:
## Generate primer usage information
# load full probes
final_pool_filename = os.path.join(pool_folder, 'final_pool_probes.fasta')
pool_records = []
with open(os.path.join(pool_folder, final_pool_filename), 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        pool_records.append(record)

In [8]:
sublibrary_probe_dict = {}
for _record in pool_records:
    _lib = _record.name.split('library_')[-1]
    if _lib not in sublibrary_probe_dict:
        sublibrary_probe_dict[_lib] = [_record]
    else:
        sublibrary_probe_dict[_lib].append(_record)
for _lib, _pbs in sublibrary_probe_dict.items():
    print(_lib, len(_pbs))

1000-mouse-genome_500 57936
1000-mouse-genome_250_1 29037
1000-mouse-genome_250_2 27746
1000-mouse-genome-short_500 29845
1000-mouse-genome-short_250_1 14988
1000-mouse-genome-short_250_2 14339


In [9]:
# find primers
primer_len=20
primer_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Primers'


sublibrary_2_primers = {}
for _lib, _pbs in sublibrary_probe_dict.items():
    #_fwd_prm = _pbs[0][:primer_len]
    #_rev_prm = _pbs[0][-primer_len:].reverse_complement()
    _primer_names = _pbs[0].name.split('_primers_[')[1].split(']')[0].split(',')
    _fwd_prm, _rev_prm = library_tools.assemble.load_primers([int(_n.split('_')[-1]) for _n in _primer_names], 
                                                             _primer_folder=primer_folder)
    
    #_fwd_prm.id, _rev_prm.id = _primer_names
    #_fwd_prm.name, _rev_prm.name = '', ''
    #_fwd_prm.description, _rev_prm.description = '', ''
    sublibrary_2_primers[_lib] = (_fwd_prm, _rev_prm)

NameError: name 'library_tools' is not defined

In [28]:
import pandas as pd

In [51]:
sublibrary_df = pd.DataFrame()
sublibrary_df['Library'] = ['CTP11-mouse-genome'] * len(sublibrary_probe_dict)
sublibrary_df['Sublibrary'] = list(sublibrary_probe_dict.keys())
sublibrary_df['Forward primer name'] = [_v[0].id for _lib, _v in sublibrary_2_primers.items()]
sublibrary_df['Forward primer seq'] = [str(_v[0].seq) for _lib, _v in sublibrary_2_primers.items()]
sublibrary_df['Reverse primer name'] = [_v[1].id for _lib, _v in sublibrary_2_primers.items()]
sublibrary_df['Reverse primer seq'] = [str(_v[1].seq) for _lib, _v in sublibrary_2_primers.items()]

sublibrary_df['Forward number'] = [int(_v[0].id.split('_')[-1]) for _lib, _v in sublibrary_2_primers.items()]
sublibrary_df['Reverse number'] = [int(_v[1].id.split('_')[-1]) for _lib, _v in sublibrary_2_primers.items()]

sublibrary_df['Sublibrary size'] = [len(_pbs) for _lib, _pbs in sublibrary_probe_dict.items()]


In [52]:
sublibrary_df.to_csv(os.path.join(pool_folder, 'sublibrary_info.csv'), index=False)

In [53]:
(sublibrary_df)

,Library,Sublibrary,Forward primer name,Forward primer seq,Reverse primer name,Reverse primer seq,Forward number,Reverse number,Sublibrary size
0,CTP11-mouse-genome,1000-mouse-genome_500,W1A03_primer_2,CCCGCAATGGCTGACAACCG,W1A10_primer_9,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG,2,9,57936
1,CTP11-mouse-genome,1000-mouse-genome_250_1,W1A07_primer_6,CGCAAACTGGTGCGGAAGGC,W1A12_primer_11,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG,6,11,29037
2,CTP11-mouse-genome,1000-mouse-genome_250_2,W1B05_primer_16,CGGGTTTCGTTGCGCACACC,W1B04_primer_15,TAATACGACTCACTATAGGGCTTGTGCATCGCGCCAAAGA,16,15,27746
3,CTP11-mouse-genome,1000-mouse-genome-short_500,W1A11_primer_10,TTGCTAGCGTGGGCCAATCG,W1B06_primer_17,TAATACGACTCACTATAGGGCCTGTGCGGAAATCGCGAGA,10,17,29845
4,CTP11-mouse-genome,1000-mouse-genome-short_250_1,W1A05_primer_4,CATTCAGCATTGCGCAACGG,W1A04_primer_3,TAATACGACTCACTATAGGGCGTTGTATGCCCTCCACGC,4,3,14988
5,CTP11-mouse-genome,1000-mouse-genome-short_250_2,W1A01_primer_0,CGGCTCGCAGCGTGTAAACG,W1A02_primer_1,TAATACGACTCACTATAGGGCATTTCAGGATCACCGGCGG,0,1,14339


# Calculate dissolve volume

for 2021.11.06 library synthesis

In [55]:
# 1000-mouse-genome_250_1
vol = 11 * 1e3 # ul
conc = 130 * 1e-9 # g/ul
num_pb = 29037
pb_len = 142-11
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.00143
4.924751179529566e-08
1.23539741156229e-12
final in ul: 123.53974115622901


In [56]:
# 1000-mouse-genome_250_2
vol = 11 * 1e3 # ul
conc = 125 * 1e-9 # g/ul
num_pb = 27746
pb_len = 142-11
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.0013750000000000001
4.955669285662799e-08
1.2431533665120898e-12
final in ul: 124.31533665120898


In [57]:
# 1000-mouse-genome_500_1
vol = 11 * 1e3 # ul
conc = 116 * 1e-9 # g/ul
num_pb = 57936
pb_len = 142-11
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.001276
2.2024302678818007e-08
5.524901772494276e-13
final in ul: 55.24901772494276


In [58]:
# 1000-mouse-genome_500_2
vol = 11 * 1e3 # ul
conc = 130 * 1e-9 # g/ul
num_pb = 57936
pb_len = 142-11
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.00143
2.4682408174537422e-08
6.191700262278068e-13
final in ul: 61.917002622780686


## Second synthesis 20220507

In [10]:
# 1000-mouse-genome_250_1
vol = 11.5 * 1e3 # ul
conc = 116 * 1e-9 # g/ul
num_pb = 29037
pb_len = 142-5.5
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.0013340000000000001
4.594138512931777e-08
1.1061137820491326e-12
final in ul: 110.61137820491327


In [11]:
# 1000-mouse-genome_250_2
vol = 11.5 * 1e3 # ul
conc = 108 * 1e-9 # g/ul
num_pb = 27746
pb_len = 142-5.5
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.001242
4.476320911122324e-08
1.077747272688872e-12
final in ul: 107.7747272688872


In [12]:
# 1000-mouse-genome_500
vol = 23 * 1e3 # ul
conc = 82.4 * 1e-9 # g/ul
num_pb = 57936
pb_len = 142-11/2
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.0018952000000000003
3.2711958022645686e-08
7.875937459180044e-13
final in ul: 78.75937459180044


In [13]:
# LPCDH
vol = 11.5 * 1e3 # ul
conc = 50 * 1e-9 # g/ul
num_pb = 14343
pb_len = 94
final_conc = 10 # nM
pb_mw = (pb_len * 303.7 + 79) # g/mol

mass_total = vol * conc
mass_per_pb = mass_total / num_pb
mol_per_pb = mass_per_pb / pb_mw
print(mass_total)
print(mass_per_pb)
print(mol_per_pb)
target_conc = final_conc * 1e-9 * 1e-6 # mol/ul
final_vol = mol_per_pb / target_conc
print("final in ul:", final_vol)

0.0005750000000000001
4.0089242139022525e-08
1.4004094812910464e-12
final in ul: 140.04094812910463
